# Pre-training Roberta on Punjab address corpus!!

In [1]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer
import torch
from transformers import AutoModelForMaskedLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from transformers import RobertaTokenizerFast

/home/uddeshya.singh/miniconda3/envs/uddeshya_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# csv_path = '/home/uddeshya.singh/DataSet_triplet/combined_cleaned_triplet_train_PUNJAB.csv'
# df = pd.read_csv(csv_path)

In [3]:
# addresses = df['cleaned_address'].tolist()
# addresses = addresses[:10000]

# data_dict = {"text": addresses}
# dataset = Dataset.from_dict(data_dict)

In [2]:
import pickle

input_pickle_path = 'DataSet_triplet/PUNJAB_5_for_lookup_without_city.pkl'

with open(input_pickle_path, 'rb') as f:
    data = pickle.load(f)

addresses = data.get('addresses', [])
lat_lngs = data.get('lat_lngs', [])

addresses_length = len(addresses)
lat_lngs_length = len(lat_lngs)

print(f"Length of addresses: {addresses_length},{addresses[-1]}")
print(f"Length of lat_lngs: {lat_lngs_length},{lat_lngs[-1]}")

Length of addresses: 1114180,wardno 1 vpo sarhala ranuan
Length of lat_lngs: 1114180,[31.2044125, 75.8583784]


In [4]:
import pickle

input_pickle_path2 = 'DataSet_triplet/combined_triplets_PUNJAB_05.1.pkl'

with open(input_pickle_path2, 'rb') as f:
    data = pickle.load(f)

addresses2 = data.get('addresses', [])
lat_lngs2 = data.get('lat_lngs', [])

addresses_length2 = len(addresses)
lat_lngs_length2 = len(lat_lngs)

print(f"Length of addresses: {addresses_length2},{addresses[-1]}")
print(f"Length of lat_lngs: {lat_lngs_length2},{lat_lngs[-1]}")

Length of addresses: 1114180,wardno 1 vpo sarhala ranuan
Length of lat_lngs: 1114180,[31.2044125, 75.8583784]


In [3]:
addresses[40:60]

[' 213 near pnb bank in ghal kalan',
 ' 218 rattian moga',
 ' 22 moga subdistrict',
 ' 224 shrma de shop wali gali',
 ' 226 gurdwara',
 ' 228 mangewala moga',
 ' 233 baba lal singh road salina',
 ' 233 kahan singh wala pasa baba lal singh gurudwara road salina',
 ' 234234 moga subdistrict',
 ' 242 daroli bhai',
 ' 244 a mangewala',
 ' 25 dadahur moga subdistrict',
 ' 254 no akal acadamy rattian salina road moga',
 ' 256 moga subdistrict',
 ' 258 gill pati',
 ' 265 darapur',
 ' 265 vpo bukkanwala moga',
 ' 278 ghall kalan agwar kushali ka',
 ' 283 jhandiana west dist moga',
 ' 293 dharamshala']

In [6]:
addresses2[40:60]

[' 211 moga subdistrict Moga District',
 ' 213 near pnb bank in ghal kalan Moga',
 ' 218 rattian moga Moga District',
 ' 22 moga subdistrict MOGA',
 ' 22 moga subdistrict Moga District',
 ' 224 shrma de shop wali gali Moga',
 ' 226 gurdwara Moga',
 ' 228 mangewala moga Moga',
 ' 233 baba lal singh road salina Moga',
 ' 233 kahan singh wala pasa baba lal singh gurudwara road salina Moga',
 ' 234234 moga subdistrict Moga',
 ' 242 daroli bhai Moga',
 ' 244 a mangewala Moga',
 ' 25 dadahur moga subdistrict Moga District',
 ' 254 no akal acadamy rattian salina road moga Moga District',
 ' 256 moga subdistrict Firozpur District',
 ' 258 gill pati Moga',
 ' 265 darapur Moga',
 ' 265 vpo bukkanwala moga Moga',
 ' 278 ghall kalan agwar kushali ka Moga District']

In [3]:
data_dict = {"text": addresses}
dataset = Dataset.from_dict(data_dict)

In [4]:
tokenizer_path = "/home/devanapalli.ravi/disk_1/paper/data/gpu_re_trained_models/lm_retrained_models/pan_india_train_threshold/pan_india_train_threshold_out_dir_5_epochs/checkpoint-375000"  
#tokenizer_path = './custom_tokenizer_punjab5_without_city_new'
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
# tokenizer = RobertaTokenizerFast.from_pretrained('./custom_tokenizer')


In [5]:
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True, num_proc=None)

Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1114180/1114180 [03:02<00:00, 6094.17 examples/s]


In [7]:
model_path = "/home/devanapalli.ravi/disk_1/paper/data/gpu_re_trained_models/lm_retrained_models/pan_india_train_threshold/pan_india_train_threshold_out_dir_5_epochs/checkpoint-375000"
model = AutoModelForMaskedLM.from_pretrained(model_path)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

In [8]:
training_args = TrainingArguments(
    output_dir='./results',          
    overwrite_output_dir=True,        
    num_train_epochs=1,               
    per_device_train_batch_size=4,   
    save_steps=10_000,                
    save_total_limit=2,
    logging_dir='./logs',            
    logging_steps=200,
    fp16=True,
)

trainer = Trainer(
    model=model,                       
    args=training_args,               
    data_collator=data_collator,       
    train_dataset=tokenized_dataset,   
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [9]:
print('hi')

hi


In [10]:
trainer.train()

/home/uddeshya.singh/miniconda3/envs/uddeshya_env/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


KeyboardInterrupt: 

## Training Pan India Model (Masked-level-modelling) with custom tokrnizer for 10,000 samples for sanity check!!

In [16]:
trainer.train()

TrainOutput(global_step=6250, training_loss=1.4751060333251953, metrics={'train_runtime': 2374.4997, 'train_samples_per_second': 21.057, 'train_steps_per_second': 2.632, 'total_flos': 6669285409898496.0, 'train_loss': 1.4751060333251953, 'epoch': 5.0})

In [17]:
trainer.train()

TrainOutput(global_step=6250, training_loss=1.1668082891845704, metrics={'train_runtime': 2374.5557, 'train_samples_per_second': 21.057, 'train_steps_per_second': 2.632, 'total_flos': 6631090329600000.0, 'train_loss': 1.1668082891845704, 'epoch': 5.0})

## Training Pan-India model with it's built-in tokenizer!!

In [24]:
trainer.train()

/home/uddeshya.singh/miniconda3/envs/uddeshya_env/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [18]:
model.save_pretrained('./trained_model_RP')
tokenizer.save_pretrained('./trained_model_RP')

('./trained_model_RP/tokenizer_config.json',
 './trained_model_RP/special_tokens_map.json',
 './trained_model_RP/vocab.json',
 './trained_model_RP/merges.txt',
 './trained_model_RP/added_tokens.json',
 './trained_model_RP/tokenizer.json')

## Training the custom tokenizer!!

In [20]:
from transformers import AutoTokenizer
from tqdm import tqdm

def training_corpus():
    for i in tqdm(range(0, len(addresses), 1000), desc="Training Tokenizer", unit="batch"):
        yield addresses[i: i + 1000]

In [21]:
old_tokenizer_path = "/home/devanapalli.ravi/disk_1/paper/data/gpu_re_trained_models/lm_retrained_models/pan_india_train_threshold/pan_india_train_threshold_out_dir_5_epochs/checkpoint-375000"  

old_tokenizer = AutoTokenizer.from_pretrained(old_tokenizer_path)


In [23]:
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus(), 52000)

Training Tokenizer: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1115/1115 [00:02<00:00, 430.32batch/s]


In [33]:
new_tokenizer.save_pretrained('./custom_tokenizer_punjab5_without_city_new')

('./custom_tokenizer_punjab5_without_city_new/tokenizer_config.json',
 './custom_tokenizer_punjab5_without_city_new/special_tokens_map.json',
 './custom_tokenizer_punjab5_without_city_new/vocab.json',
 './custom_tokenizer_punjab5_without_city_new/merges.txt',
 './custom_tokenizer_punjab5_without_city_new/added_tokens.json',
 './custom_tokenizer_punjab5_without_city_new/tokenizer.json')